This notebook will perform tests where:

$$\rho_{x,y \cdot \mathbf{z}}$$

Where $|\mathbf{z}| = n$ and $n = 1, 2, 3...$

In [98]:
from scipy.stats import chi2_contingency
from scipy import stats
import scipy
import numpy as np
import random
from scipy.stats import norm
from sklearn.metrics.pairwise import rbf_kernel, linear_kernel, pairwise_kernels
from sklearn.kernel_approximation import Nystroem
from sklearn import preprocessing

from itertools import combinations, permutations, chain

In [5]:
def fast_cor1(X, y):
    """
    As the dependency is a single variable there 
    is no need to loop through
    we can use a kernel trick/approximation to solve it quickly.
    
    We do not care if things are perfectly negatively correlated here
    """
    X_all = np.hstack([X, y.reshape(-1, 1)]).T
    #X_all = X_all - X_all.mean(axis=0)
    
    # calculate correlation
    if X_all.shape[0] < 1000:
        K = pairwise_kernels(X_all, metric='cosine')
    else:
        K = Nystroem('cosine').fit_transform(X_all)
    
    #k_I = np.eye(K.shape[0])
    #cov = K.dot(np.linalg.pinv(k_I - K))
    d_inv = np.sqrt(np.diag(np.diag(K)))
    corr = d_inv.dot(K).dot(d_inv)
    return corr


In [6]:
# calculate the partial correlation 
# using the info from wikipedia
def fast_partial_cor1(X, y):
    """
    Calculate the partial correlation for order 1
    """
    cor_f = fast_cor(X, y)
    # now take out the last row and column...
    cor_dim = cor_f.shape[0]-1
    cor_x = cor_f[:, :cor_dim][:cor_dim, :]
    cor_z = cor_f[cor_dim, :cor_dim].flatten()
    
    # calculate new correlation matrix
    cor_m = np.ones((cor_x.shape[1], cor_x.shape[1]))
    y_reshape = y.reshape(-1, 1)
    tri_idx = np.triu_indices(cor_m.shape[1], 0)
    for i, j in zip(tri_idx[0].flatten(), tri_idx[1].flatten()):
        corr = (cor_x[i, j] - cor_z[i]*cor_z[j])
        corr = corr/(np.sqrt(1-(cor_z[i]*cor_z[i]))*np.sqrt(1-(cor_z[j]*cor_z[j])))
        cor_m[i, j] = corr
        cor_m[j, i] = corr
    return cor_m #, cor_x 

In [ ]:
def fast_cor1(X, y):
    """
    As the dependency is a single variable there 
    is no need to loop through
    we can use a kernel trick/approximation to solve it quickly.
    
    We do not care if things are perfectly negatively correlated here
    """
    X_all = np.hstack([X, y.reshape(-1, 1)]).T
    #X_all = X_all - X_all.mean(axis=0)
    
    # calculate correlation
    if X_all.shape[0] < 1000:
        K = pairwise_kernels(X_all, metric='cosine')
    else:
        K = Nystroem('cosine').fit_transform(X_all)
    
    #k_I = np.eye(K.shape[0])
    #cov = K.dot(np.linalg.pinv(k_I - K))
    d_inv = np.sqrt(np.diag(np.diag(K)))
    corr = d_inv.dot(K).dot(d_inv)
    return corr

In [14]:
def fast_cor(X_all):
    # calculate correlation
    X_scaled = preprocessing.scale(X_all)
    if X_all.shape[0] < 1000:
        K = pairwise_kernels(X_scaled.T, metric='cosine')
        return K
    else:
        K = Nystroem('cosine').fit_transform(X_scaled.T)
        d_inv = np.sqrt(np.diag(np.diag(K)))
        corr = d_inv.dot(K).dot(d_inv)
        return corr

In [55]:
def fisher_test(cor_m, N, z_n):
    cor_m = np.minimum(cor_m, 0.9999)
    cor_m = np.maximum(cor_m, -0.9999)
    z_score = 0.5*np.log((1+cor_m)/(1-cor_m))
    test_stat = np.sqrt(N - z_n -3) * np.abs(z_score)
    p_val = 1-scipy.stats.norm.cdf(test_stat)
    return p_val

In [56]:
x1 = np.random.normal(size=(10,))
c = np.random.normal(size=(10,))
x_dat = np.random.normal(size=(10, 3))
x_all = np.hstack([x1.reshape(-1, 1), 
                  c.reshape(-1, 1), 
                  x_dat])

In [95]:
# pull out vectors for x1, and c...
x_cor = fast_cor(x_all)
x_cor_inv = np.linalg.pinv(x_cor)
x1_cor = x_cor[:, 0]
c_cor = x_cor[:, 1]    

In [106]:
x_cor_inv

array([[ 1.51922542, -0.26146002, -0.3971878 ,  0.46691053, -0.39883903],
       [-0.26146002,  2.71434436, -0.47492249, -0.47380797, -2.03720088],
       [-0.3971878 , -0.47492249,  1.24072417,  0.06957941,  0.66159051],
       [ 0.46691053, -0.47380797,  0.06957941,  1.21856148,  0.27227746],
       [-0.39883903, -2.03720088,  0.66159051,  0.27227746,  2.78708648]])

In [108]:
np.linalg.pinv(x_cor[:, :3][:3, :])

array([[ 1.24745528, -0.43636697, -0.30461206],
       [-0.43636697,  1.16191862,  0.0098028 ],
       [-0.30461206,  0.0098028 ,  1.0836572 ]])

In [104]:
def powerset(iterable, max_size=3):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, max_size+1))

In [126]:
def partial_cor(x1, c, x_dat, x1_name, x_dat_names, max_search_depth=3):
    max_search_depth = min(max_search_depth, len(x_dat_names))
    x_n = x_dat.shape[0]
    x_all = np.hstack(
        [x1.reshape(-1, 1), 
         c.reshape(-1, 1), 
         x_dat])
    x_cor = fast_cor(x_all)
    partial_cor = []
    for cond in list(powerset(range(2, x_cor.shape[1]), max_search_depth)):
        z_n = len(cond)
        idx = [0, 1] + list(cond)
        V = x_cor[idx, :][:, idx]
        V_inv = np.linalg.pinv(V)
        cor = -V_inv[0][1]/(np.sqrt(V_inv[0][0]*V_inv[1][1]))
        partial_cor.append({
            'var': x1_name,
            'cor': cor,
            'pval': fisher_test(cor, x_n, z_n),
            'cond': set([x_dat_names[x-2] for x in cond])
        })
    return partial_cor

In [127]:
partial_cor(x1, c, x_dat, 'x1', ['x2', 'x3', 'x4'], 2)

[{'cond': {'x2'},
  'cor': 0.36245284974583053,
  'pval': 0.17616275762972311,
  'var': 'x1'},
 {'cond': {'x3'},
  'cor': 0.42377171743350006,
  'pval': 0.13396194549508977,
  'var': 'x1'},
 {'cond': {'x4'},
  'cor': 0.13874771241880826,
  'pval': 0.36615003962054271,
  'var': 'x1'},
 {'cond': {'x2', 'x3'},
  'cor': 0.41314711704975554,
  'pval': 0.16291974043895752,
  'var': 'x1'},
 {'cond': {'x2', 'x4'},
  'cor': 0.043395428033868248,
  'pval': 0.46132503624746235,
  'var': 'x1'},
 {'cond': {'x3', 'x4'},
  'cor': 0.22022137005973111,
  'pval': 0.30831560887057152,
  'var': 'x1'}]

In [109]:
# using matrix inverstion...
partial_cor = []
for cond in list(powerset(range(2, x_cor.shape[1]))):
    idx = [0, 1] + list(cond)
    V = x_cor[idx, :][:, idx]
    V_inv = np.linalg.pinv(V)
    cor = -V_inv[0][1]/(np.sqrt(V_inv[0][0]*V_inv[1][1]))
    partial_cor.append({
        'cor': cor,
        'cond': set(cond)
    })

In [110]:
partial_cor

[{'cond': {2}, 'cor': 0.36245284974583053},
 {'cond': {3}, 'cor': 0.42377171743350006},
 {'cond': {4}, 'cor': 0.13874771241880826},
 {'cond': {2, 3}, 'cor': 0.41314711704975554},
 {'cond': {2, 4}, 'cor': 0.043395428033868248},
 {'cond': {3, 4}, 'cor': 0.22022137005973111},
 {'cond': {2, 3, 4}, 'cor': 0.12875428999801561}]

In [58]:
def partial_cor_single(pxy, pxz, pzy):
    a = pxy - (pxz*pzy)
    b = np.sqrt(1-(pxz*pxz))*np.sqrt(1-(pzy*pzy))
    return a/b

In [67]:
[x for x in list(combinations(list(range(2, x_cor.shape[1]))+[0], 2)) if x[0] == 0 or x[1] == 0]

[(2, 0), (3, 0), (4, 0)]

In [129]:
set([1]) == set([1, 2])

False

In [91]:
order1 = []
x_n = x_all.shape[0]
z_n = 1

for base0, base1, cond0 in list(permutations(range(x_cor.shape[1]), 3)):
    if cond0 == 1 or base0 > base1:
        # do not condition on label
        continue
    cor = partial_cor_single(x_cor[base0][base1], x_cor[base0][cond0], x_cor[base1][cond0])
    pval = fisher_test(cor, x_n, z_n)
    order1.append(
        {'cor_pair': set([base0, base1]), 
         'condition': set([cond0]),
         'cor': cor, 
         'pval': pval}
    )

In [92]:
#order 2
x_n = x_all.shape[0]
z_n = 2

for base0, base1, cond0, cond1 in list(permutations(range(x_cor.shape[1]), 4)):
    if cond0 == 1 or cond1 == 1 or base0 > base1:
        # do not condition on label
        continue
    pxy = [x['cor'] for x in order1 if base0 in x['cor_pair'] and base1 in x['cor_pair'] and cond1 in x['condition']][0]
    pxz = [x['cor'] for x in order1 if base0 in x['cor_pair'] and cond0 in x['cor_pair'] and cond1 in x['condition']][0]
    pzy = [x['cor'] for x in order1 if base1 in x['cor_pair'] and cond0 in x['cor_pair'] and cond1 in x['condition']][0]
    
    cor = partial_cor_single(pxy, pxz, pzy)
    pval = fisher_test(cor, x_n, z_n)
    order1.append(
        {'cor_pair': set([base0, base1]), 
         'condition': set([cond0, cond1]),
         'cor': cor, 
         'pval': pval}
    )


In [93]:
x_n = x_all.shape[0]
z_n = 3

for base0, base1, cond0, cond1, cond2 in list(permutations(range(x_cor.shape[1]), 5)):
    if cond0 == 1 or cond1 == 1 or cond2 == 1 or base0 > base1:
        # do not condition on label
        continue
    pxy = [x['cor'] for x in order1 if base0 in x['cor_pair'] and base1 in x['cor_pair'] and cond1 in x['condition'] and cond2 in x['condition']][0]
    pxz = [x['cor'] for x in order1 if base0 in x['cor_pair'] and cond0 in x['cor_pair'] and cond1 in x['condition'] and cond2 in x['condition']][0]
    pzy = [x['cor'] for x in order1 if base1 in x['cor_pair'] and cond0 in x['cor_pair'] and cond1 in x['condition'] and cond2 in x['condition']][0]
    
    cor = partial_cor_single(pxy, pxz, pzy)
    pval = fisher_test(cor, x_n, z_n)
    order1.append(
        {'cor_pair': set([base0, base1]), 
         'condition': set([cond0, cond1, cond2]),
         'cor': cor, 
         'pval': pval}
    )

In [94]:
order1

[{'condition': {2},
  'cor': 0.36245284974583025,
  'cor_pair': {0, 1},
  'pval': 0.17616275762972344},
 {'condition': {3},
  'cor': 0.42377171743350051,
  'cor_pair': {0, 1},
  'pval': 0.13396194549508955},
 {'condition': {4},
  'cor': 0.13874771241880826,
  'cor_pair': {0, 1},
  'pval': 0.36615003962054271},
 {'condition': {3},
  'cor': 0.26049660805350633,
  'cor_pair': {0, 2},
  'pval': 0.25683474649445936},
 {'condition': {4},
  'cor': 0.35197985484681532,
  'cor_pair': {0, 2},
  'pval': 0.1838787764761981},
 {'condition': {2},
  'cor': -0.32616836242181751,
  'cor_pair': {0, 3},
  'pval': 0.20348526904623077},
 {'condition': {4},
  'cor': -0.33931738775276887,
  'cor_pair': {0, 3},
  'pval': 0.19339430333635321},
 {'condition': {2},
  'cor': 0.4359254165189706,
  'cor_pair': {0, 4},
  'pval': 0.12623406254056957},
 {'condition': {3},
  'cor': 0.38297302083451046,
  'cor_pair': {0, 4},
  'pval': 0.16146254921120318},
 {'condition': {0},
  'cor': -0.008736070355021771,
  'cor_pair'